In [0]:
import torch
import torch.nn as nn  #neural networks 
import torchvision # used to visuallising things editing data
import torchvision.transforms as transforms # transform operation like croping 

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# .ToTenser will convert numpy to tensor----
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:02, 3393432.36it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 49502.20it/s]                           
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:01, 827772.40it/s]                             
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


8192it [00:00, 18449.94it/s]            

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


In [0]:
# Hyper-parameters 
input_size = 784
hidden_size = 500#number of nurons in second layer
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [0]:
# Data loader-----used to pass dataset in parts and not all at once-----it is used to shuffle data too
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

#test should not be shuffled as shuffling will lead to wrong submission in kaggle also !!!! 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()#setting the optimiser values to 0
        loss.backward() #backward pass step
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.6037
Epoch [1/5], Step [200/600], Loss: 0.3943
Epoch [1/5], Step [300/600], Loss: 0.1824
Epoch [1/5], Step [400/600], Loss: 0.1939
Epoch [1/5], Step [500/600], Loss: 0.1070
Epoch [1/5], Step [600/600], Loss: 0.1541
Epoch [2/5], Step [100/600], Loss: 0.1814
Epoch [2/5], Step [200/600], Loss: 0.1765
Epoch [2/5], Step [300/600], Loss: 0.0840
Epoch [2/5], Step [400/600], Loss: 0.0699
Epoch [2/5], Step [500/600], Loss: 0.0723
Epoch [2/5], Step [600/600], Loss: 0.0777
Epoch [3/5], Step [100/600], Loss: 0.0548
Epoch [3/5], Step [200/600], Loss: 0.0951
Epoch [3/5], Step [300/600], Loss: 0.0629
Epoch [3/5], Step [400/600], Loss: 0.0750
Epoch [3/5], Step [500/600], Loss: 0.0498
Epoch [3/5], Step [600/600], Loss: 0.0346
Epoch [4/5], Step [100/600], Loss: 0.0185
Epoch [4/5], Step [200/600], Loss: 0.0582
Epoch [4/5], Step [300/600], Loss: 0.1086
Epoch [4/5], Step [400/600], Loss: 0.0482
Epoch [4/5], Step [500/600], Loss: 0.0139
Epoch [4/5], Step [600/600], Loss:

In [7]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.67 %


In [0]:
torch.save(model.state_dict(), 'model.ckpt')


CNN

In [0]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)


In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1248
Epoch [1/5], Step [200/600], Loss: 0.1975
Epoch [1/5], Step [300/600], Loss: 0.0464
Epoch [1/5], Step [400/600], Loss: 0.0672
Epoch [1/5], Step [500/600], Loss: 0.0765
Epoch [1/5], Step [600/600], Loss: 0.0180
Epoch [2/5], Step [100/600], Loss: 0.0989
Epoch [2/5], Step [200/600], Loss: 0.0113
Epoch [2/5], Step [300/600], Loss: 0.0698
Epoch [2/5], Step [400/600], Loss: 0.0116
Epoch [2/5], Step [500/600], Loss: 0.0390
Epoch [2/5], Step [600/600], Loss: 0.1287
Epoch [3/5], Step [100/600], Loss: 0.0540
Epoch [3/5], Step [200/600], Loss: 0.0594
Epoch [3/5], Step [300/600], Loss: 0.0038
Epoch [3/5], Step [400/600], Loss: 0.0077
Epoch [3/5], Step [500/600], Loss: 0.0475
Epoch [3/5], Step [600/600], Loss: 0.0199
Epoch [4/5], Step [100/600], Loss: 0.0149
Epoch [4/5], Step [200/600], Loss: 0.0333
Epoch [4/5], Step [300/600], Loss: 0.0065
Epoch [4/5], Step [400/600], Loss: 0.0273
Epoch [4/5], Step [500/600], Loss: 0.0013
Epoch [4/5], Step [600/600], Loss:

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [19]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

def W_init(shape,name=None):
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

def b_init(shape,name=None):

    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)


optimizer = Adam(0.00006)

siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

Using TensorFlow backend.


TypeError: ignored